In [ ]:
# import necessary libraries
# !pip install PyPDF2
# !pip install pandas as pd
# !pip install textstat
# !pip install docx2txt
# !pip install spacy
# !pip install openai
# !pip install python-docx
# !pip install numpy
# !pip install matplotlib
# !pip install scipy

import PyPDF2
import pandas as pd
import textstat
import docx2txt
import spacy
import openai
from docx import Document
openai.api_key = ""
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [27]:
#PRE CHAT GPT

In [28]:
# initialize PRE GPT dataframe to store results for each file
df = pd.DataFrame(columns=['file', 'flesch_reading_ease', 'flesch_grade', 'gunning_fog'])

In [29]:
#POST GPT

In [30]:
#PROMPT ONE

def getGPTResponse(inputText):
    customPrompt = f"""Reword without summarizing for someone with an eighth-grade reading level,
                       do not change titles or headers, keep format. {inputText}"""
    response = openai.Completion.create(model="text-davinci-003",
                                        prompt=customPrompt,
                                        max_tokens=3000)
    print(response.choices[0].text)
    return response.choices[0].text

In [31]:
# #PROMPT TWO

# def getGPTResponse(inputText):
#     customPrompt = f"""Decrease the Flesch-Kincaid Grade Level score without decreasing number of words or changing titles,
#                        keep the format.{inputText}"""
#     response = openai.Completion.create(model="text-davinci-003",
#                                         prompt=customPrompt, 
#                                         max_tokens=3000)
#     print(response.choices[0].text)
#     return response.choices[0].text


In [32]:
# #PROMPT THREE

# def getGPTResponse(inputText):
#     customPrompt = f"""Increase the overall readability do not decrease the number of words, summarize,
#                         or change any titles or headers, keep the format. {inputText}"""
#     response = openai.Completion.create(model="text-davinci-003",
#                                         prompt=customPrompt, 
#                                         max_tokens=3000)
#     print(response.choices[0].text)
#     return response.choices[0].text

In [33]:
#call split text we will return responses
def splitText(inputText):
    lines = inputText.split("\n")
    half = len(lines)//2

# generate new text for first half
    firstHalf = '\n'.join(lines[:half])
    if len(firstHalf) > 1500:
        return splitText(firstHalf)
    else:
        newText = getGPTResponse(firstHalf)
        return newText

# generate new text for second half
    secondHalf = '\n'.join(lines[half:])
    if len(secondHalf) > 1500:
        return splitText(secondHalf)
    else:
        newText = getGPTResponse(secondHalf)
        return newText

In [34]:
# initialize dataframe to store results for each file
df2 = pd.DataFrame(columns=['file','flesch_reading_ease', 'flesch_grade', 'gunning_fog'])

In [36]:
def get_response(file, inputText):
    post_GPT_text = ""
    if len(inputText) > 1500: # text is too long for chat GPT to handle, we need to split it
        post_GPT_text += splitText(inputText) #adding the split text response 
    else:
        post_GPT_text = getGPTResponse(inputText)
    flesch_reading_ease = textstat.flesch_reading_ease(post_GPT_text)
    flesch_grade = textstat.flesch_kincaid_grade(post_GPT_text)
    gunning_fog = textstat.gunning_fog(post_GPT_text) 
    
    #append stats for this file to existing dataframe
    global df2
    df2 = pd.concat([df2, pd.DataFrame({'file': [f'ICF_{file}.pdf'],'flesch_reading_ease': [flesch_reading_ease],  
                                        'flesch_grade': [flesch_grade],'gunning_fog':[gunning_fog]})])

In [ ]:
files = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,
 40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,
 77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,
 110, 111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,
 137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,
 164,165,166,167,168,169,170,171,172,173,174,175]

for file in files:
    reader = PyPDF2.PdfReader(f'C:/Users/Taylo/OneDrive/Desktop/BREAST CANCER ICF PDF/ICF_{file}.pdf')
    pages = reader.pages
    text = ""
    for page in pages:
        text += page.extract_text()
    flesch_reading_ease = textstat.flesch_reading_ease(text)
    flesch_grade = textstat.flesch_kincaid_grade(text)
    gunning_fog = textstat.gunning_fog(text)
 
    # append stats for this file to existing dataframe
    df = pd.concat([df, pd.DataFrame({'file': [f'ICF_{file}.pdf'], 'flesch_reading_ease': [flesch_reading_ease], 
                                      'flesch_grade': [flesch_grade], 'gunning_fog':[gunning_fog]})])
 
    #pull response , do the calculation , add to df
    get_response(file, text)


 
Someone with an eighth-grade reading level: 
This is a document about taking part in research called "Molecular Mechanisms of Clinical Resistance to Targeted Therapy Among Patients with Breast Cancer". The person in charge of the research is called Sarat Chandarlapaty, MD, PhD. If the person taking part in the study is a minor or someone with a cognitive impairment, you can give consent on their behalf. In this document, "you" means your child, or the person with the cognitive impairment, if you are their Legally Authorized Representative (LAR).

Effects of an Exercise Intervention on Physical Activity during Chemotherapy 
for Patients with Early Stage Breast Cancer
Do you want to be in this study?
This form is about a research study. Researchers (people who do studies) want to find out if an exercise program can help people with early stage breast cancer who are having chemotherapy. Before you decide if you want to take part, you need to understand why the research is being done an

In [ ]:
PRE CHAT GPT RESULTS
df.head(175)

In [ ]:
#POST CHAT GPT RESULTS
#df2.head(175)

In [28]:
#df.to_csv("C:/Users/Taylo/OneDrive/Desktop/pre.csv", index=False )

In [15]:
#df2.to_csv("C:/Users/Taylo/OneDrive/Desktop/post3.csv", index=False )